# Language modelling and Text classification

### We try to analayze some textual data from the corpus and then classify the telugu texts based on their topic

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
import seaborn as sns
import os


In [ ]:
train_path = "../input/telugu-nlp/telugu_news/train_telugu_news.csv"

telugu_news_df = pd.read_csv(train_path)
telugu_news_df

# Text pre-processing

In [ ]:
telugu_news_df.isna().sum()

In [ ]:
telugu_news_df[telugu_news_df["heading"].isna() == True]

In [ ]:
del telugu_news_df["heading"]
del telugu_news_df["SNo"]

In [ ]:
telugu_news_df["topic"].unique()

In [ ]:
topic_dic = {}

c = 0
for un in telugu_news_df["topic"].unique():
    if un not in topic_dic:
        topic_dic[un] = c
        c += 1
        
topic_dic

In [ ]:
inv_topic_dict = {v: k for k, v in topic_dic.items()}

In [ ]:
inv_topic_dict


In [ ]:
def func_topic(s):
    return topic_dic[s]

telugu_news_df["topic"] = telugu_news_df["topic"].apply(func_topic)

In [ ]:
def get_count(df):
    
    unvals = list(df.unique())
    op = [0]*len(unvals)
    
    i = 0
    for un in unvals:
        op[i] = df[df == un].shape[0]
        i += 1
        
    return [list(unvals),op]

cont = get_count(telugu_news_df["topic"])
clables = cont[0]

labels = []
for c in clables:
    labels.append(inv_topic_dict[c])

plt.figure(figsize=(10,8))
plt.title("Distribution of the telugu news toics", fontsize = 14.5)
plt.style.use('seaborn-colorblind')
plt.pie(np.array(cont[1]), labels=labels, autopct='%1.2f%%', shadow=True)
plt.show()

In [ ]:
date_df = telugu_news_df["date"]
del telugu_news_df["date"]

Installing the dependancies and the indic libraries for telugu text processing

In [ ]:
pip install indic-nlp-library

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git

In [ ]:
import sys
from indicnlp import common

# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"indic_nlp_resources"

# Add library to Python path
sys.path.append(r'{}\src'.format(INDIC_NLP_LIB_HOME))

# Set environment variable for resources folder
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
from indicnlp.tokenize import sentence_tokenize

indic_string = telugu_news_df["body"][0]
# Split the sentence, language code "hi" is passed for hingi
sentences=sentence_tokenize.sentence_split(indic_string, lang='te')

# print the sentences
for t in sentences:
    print(t)

In [ ]:
telugu_news_df["body_processed"] = telugu_news_df["body"].str.replace('\u200c', '')
telugu_news_df["body_processed"] = telugu_news_df["body_processed"].str.replace('\n', '')
telugu_news_df["body_processed"] = telugu_news_df["body_processed"].str.replace('\t', '')
telugu_news_df["body_processed"] = telugu_news_df["body_processed"].str.replace('\xa0', '')

In [ ]:
PUNCT


In [ ]:
PUNCT = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT))

In [ ]:
telugu_news_df["body_processed"][6665]

In [ ]:
from indicnlp.tokenize import sentence_tokenize


tot_telugu_text1 = ""

for t in telugu_news_df["body_processed"]:
  tot_telugu_text1 += t

# tot_sentances_telugu = []

tot_sentences = sentence_tokenize.sentence_split(tot_telugu_text1, lang='te')
print(len(tot_sentences))

In [ ]:
telugu_news_df["body_processed"] = telugu_news_df["body_processed"].apply(lambda text: remove_punctuation(text))

In [ ]:
# telugu_news_df["body_processed"][0]
del telugu_news_df["body"]

In [ ]:
tot_telugu_text = ""

c = 1
for t in telugu_news_df["body_processed"]:
  tot_telugu_text += t
  c += 1

print(c)

In [ ]:
len(tot_telugu_text)

## Tokenizing and building a vocabulary for the text

In [ ]:
from indicnlp.tokenize import indic_tokenize  

vocab_dic  = {}
tokenized_text = []
heap_arr = []

for t in indic_tokenize.trivial_tokenize(tot_telugu_text):
    tokenized_text.append(t)
    heap_arr.append(len(vocab_dic))

    if t not in vocab_dic:
      vocab_dic[t] = 1

    else:
      vocab_dic[t] += 1

In [ ]:
len(vocab_dic), len(tokenized_text)

In [ ]:
len(heap_arr)

In [ ]:
from nltk.probability import FreqDist 

freq_dist = FreqDist(vocab_dic)

**Heaps Law**

In linguistics, Heaps' law (also called Herdan's law) is an empirical law which describes the number of distinct words in a document (or set of documents) as a function of the document length (so called type-token relation). It can be formulated as

-> V_R(n) = K*(n^β) 

where V_R is the number of distinct words in an instance text of size n. K and β are free parameters determined empirically. With English text corpora, typically K is between 10 and 100, and β is between 0.4 and 0.6. 

In [ ]:
plt.figure(figsize = (15,6))
plt.plot(heap_arr)
plt.title("Heap's law graph")
plt.xlabel("Vocabulary size")
plt.ylabel("No.of words in the corpus (text size)")
plt.show()

In [ ]:
vocab_dic_sorted = {k: v for k, v in sorted(vocab_dic.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
top_k_words = []

k = 50
c = 0

for key in vocab_dic_sorted.keys():

  if c == k:
    break

  top_k_words.append([key, vocab_dic_sorted[key]])
  c += 1

In [ ]:
top_k_words_df = pd.DataFrame(top_k_words)
top_k_words_df.columns = ["word", "freq"]
top_k_words_df.head()

In [ ]:
plt.figure(figsize = (15,6))
plt.plot(top_k_words_df["freq"])
plt.title("Top 50 words in the telugu corpus")
plt.xlabel("No.of words")
plt.ylabel("Frequency of the word")
# plt.xlim([0,k])
plt.show()

**NOTE**

**Here, we are not labelling words on the x-axis as the unicode representation in matplotlib is not clear.**

In [ ]:
telugu_words = list(vocab_dic_sorted.keys())

In [ ]:
tot_sentences_proc = []

for sen in tot_sentences:
  tot_sentences_proc.append(remove_punctuation(sen))

In [ ]:
tot_sentences_proc[0]

# N-Gram analysis of the entire corpus

In [ ]:
from nltk.util import ngrams

bigrams_telugu_vocab = {}

for sen in tot_sentences_proc:
  sen = '<s> '+ sen +  ' </s>'
  
  tokens = sen.split()
  bigram = list(ngrams(tokens, 2)) 

  for big in bigram:
    if tuple(big) not in bigrams_telugu_vocab:
      bigrams_telugu_vocab[tuple(big)] = 1

    else:
      bigrams_telugu_vocab[tuple(big)] += 1

In [ ]:
bigrams_telugu_vocab = {k: v for k, v in sorted(bigrams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
print("Total no.of unique bi-grams :- ", len(bigrams_telugu_vocab))

In [ ]:
k = 15
print("Top" ,k ,"most-occuring bi-grams in the corpus are\n")
c = 0
for key in bigrams_telugu_vocab:
  if c == k:
    break
  print(key , " -> ", bigrams_telugu_vocab[key])
  c += 1

In [ ]:
trigrams_telugu_vocab = {}

for sen in tot_sentences_proc:
  sen = '<s> '+ sen +  ' </s>'
  
  tokens = sen.split()
  trigram = list(ngrams(tokens, 3)) 

  for trig in trigram:
    if tuple(trig) not in trigrams_telugu_vocab:
      trigrams_telugu_vocab[tuple(trig)] = 1

    else:
      trigrams_telugu_vocab[tuple(trig)] += 1

trigrams_telugu_vocab = {k: v for k, v in sorted(trigrams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
k = 15
print("Top" ,k ,"most-occuring tri-grams in the corpus are\n")
c = 0
for key in trigrams_telugu_vocab:
  if c == k:
    break
  print(key , " -> ", trigrams_telugu_vocab[key])
  c += 1

In [ ]:
print("Total no.of unique tri-grams :- ", len(trigrams_telugu_vocab))

In [ ]:
four_grams_telugu_vocab = {}

for sen in tot_sentences_proc:
  sen = '<s> '+ sen +  ' </s>'
  
  tokens = sen.split()
  fourgram = list(ngrams(tokens, 4)) 

  for fourg in fourgram:
    if tuple(fourg) not in four_grams_telugu_vocab:
      four_grams_telugu_vocab[tuple(fourg)] = 1

    else:
      four_grams_telugu_vocab[tuple(fourg)] += 1

four_grams_telugu_vocab = {k: v for k, v in sorted(four_grams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
k = 15
print("Top" ,k ,"most-occuring 4-grams in the corpus are\n")
c = 0
for key in four_grams_telugu_vocab:
  if c == k:
    break
  print(key , " -> ", four_grams_telugu_vocab[key])
  c += 1

In [ ]:
print("Total no.of unique 4-grams :- ", len(four_grams_telugu_vocab))

In [ ]:
five_grams_telugu_vocab = {}

for sen in tot_sentences_proc:
  tokens = indic_tokenize.trivial_tokenize(sen)
  fivegram = list(ngrams(tokens, 5)) 

  for fiveg in fivegram:
    if tuple(fiveg) not in five_grams_telugu_vocab:
      five_grams_telugu_vocab[tuple(fiveg)] = 1

    else:
      five_grams_telugu_vocab[tuple(fiveg)] += 1

five_grams_telugu_vocab = {k: v for k, v in sorted(five_grams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
k = 30
print("Top" ,k ,"most-occuring 5-grams in the corpus are\n")
c = 0
for key in five_grams_telugu_vocab:
  if c == k:
    break
  print(key , " -> ", five_grams_telugu_vocab[key])
  c += 1

In [ ]:
print("Total no.of unique 5-grams :- ", len(five_grams_telugu_vocab))

In [ ]:
ngrams_count = []

ngrams_count.append(len(vocab_dic_sorted))
ngrams_count.append(len(bigrams_telugu_vocab))
ngrams_count.append(len(trigrams_telugu_vocab))
ngrams_count.append(len(four_grams_telugu_vocab))
ngrams_count.append(len(five_grams_telugu_vocab))

In [ ]:
ngrams_count

In [ ]:
plt.figure(figsize = (10,5))
plt.bar(x = ["1-gram","2-gram","3-gram","4-gram","5-gram"], height = ngrams_count)
plt.ylabel("No.of unique n-grams")

Data sparsity is more of an issue in NLP than in other machine learning fields because we typically deal with large vocabularies where it is impossible to have enough data to actually observe examples of all the things that people can say. There will be many real phrases that we will just never see in the training data. 

# Language models
Building models to classify the text based on the topic of the text

**Using n grams and laplace smoothening**

In [ ]:
telugu_news_df["topic"].unique()

In [ ]:
from nltk.util import pad_sequence

In [ ]:
pip install indic-nlp-library

In [ ]:
from indicnlp.tokenize import sentence_tokenize


# function to build a n-gram vocabulary
def build_n_gram_vocab(n, tot_sentences_proc):

  if n > 1:
    n_grams_telugu_vocab = {}

    for sen in tot_sentences_proc:
      sen =  '<s> '*(n-1) + sen + ' </s>'*(n-1)
      tokens = sen.split()
      ngram = list(ngrams(tokens, n))
      for ngm in ngram:
        if tuple(ngm) not in n_grams_telugu_vocab:
          n_grams_telugu_vocab[tuple(ngm)] = 1

        else:
          n_grams_telugu_vocab[tuple(ngm)] += 1

    n_grams_telugu_vocab = {k: v for k, v in sorted(n_grams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}


  else:
    n_grams_telugu_vocab = {}

    for sen in tot_sentences_proc:
      sen =  '<s> '*(n-1) + sen + ' </s>'*(n-1)
      tokens = sen.split()
      ngram = list(ngrams(tokens, 1)) 

      for ngm in ngram:
        if ngm not in n_grams_telugu_vocab:
          n_grams_telugu_vocab[ngm] = 1

        else:
          n_grams_telugu_vocab[ngm] += 1

    n_grams_telugu_vocab = {k: v for k, v in sorted(n_grams_telugu_vocab.items(), key=lambda item: item[1], reverse = True)}

  return n_grams_telugu_vocab

# function to get the vocabulary of the text of a class specified
def get_class_vocab(class_id, n_grams = 1):
  tot_text_class = ""

  for t in telugu_news_df[telugu_news_df["topic"] == class_id]["body_processed"]:
    tot_text_class += t

  tot_sentances_class = sentence_tokenize.sentence_split(tot_text_class, lang='te')

  vocab = build_n_gram_vocab(n_grams, tot_sentances_class)
    
  return vocab

In [ ]:
for i in inv_topic_dict:
    print(len(get_class_vocab(i, 2)))

In [ ]:
for i in inv_topic_dict:
    print(len(get_class_vocab(i))

In [ ]:
cute = "ఈ వరల్డ్ కప్ మెగా టోర్నీలో మొదటి నుంచి అద్భుత ప్రదర్శన కనబరుస్తూ వరుస విజయాలు నమోదు చేసిన భారత జట్టు. కాంగ్రెస్ ఉపాధ్యక్షుడు రాహుల్ అమెరికా అధ్యక్షుడు డొనాల్డ్ ట్రంప్ తో చర్చించారు. మార్కెట్ వాస్తవిక కదలికల ఆధారంగా ఇచ్చిన సూచన ట్రేడింగ్కు ప్రారంభ స్థాయి కీలక. స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటికీ కుటుంబంతో గడిపే అవకాశాన్ని ఏ మాత్రం వదులుకోవడ. ఆర్థికవేత్త బెంగుళూరు ఐఐఎం రిటైర్డ్ అధ్యాపకుడు వైద్యనాథన్"

In [ ]:
cute2 = sentence_tokenize.sentence_split(cute, lang = 'te')

In [ ]:
cute2

In [ ]:
# function to get the probabity of the input text to be belonging to a specific class
def get_prob_class_ngram(input_text, vocab, n_grams, class_id, laplace_param = 1):
  total_tokens = len(vocab)
  Prob_input = 1
  tokens = input_text.split()
  vocab_n_1 = get_class_vocab(class_id, n_grams-1)
  if n_grams > 1:
    ngram = list(ngrams(tokens, n_grams))
    for ngm in ngram:

      prior_ngm = tuple(ngm[:n_grams-1])
      if prior_ngm in vocab_n_1:
        count_denom = vocab_n_1[prior_ngm]
      else:
        count_denom = 1

      if tuple(ngm) in vocab:
        count_w = vocab[tuple(ngm)]
      else:
        count_w = 0
      Prob_input *= ((count_w + laplace_param)/(count_denom+(laplace_param*len(vocab_n_1))))

  else:
    tokens = input_text.split()
    ngram = list(ngrams(tokens, 1))

    for ngm in ngram:
      if ngm in vocab:
        count_w = vocab[ngm]
      else:
        count_w = 0

      prob = count_w/total_tokens
      Prob_input *= count_w/total_tokens

  return Prob_input

def get_max_ind(arr):

  n = len(arr)

  max_ind = 0
  max_prob = arr[0]

  for i in range(1,n):

    if arr[i] > max_prob:
      max_ind = i
      max_prob = arr[i]

  return max_ind, max_prob


# Function to classify the given input text using the n_grams model
def classify_text(text, n_grams, laplace_param = 0):

  cls0_vocab = get_class_vocab(0, n_grams)
  cls1_vocab = get_class_vocab(1, n_grams)
  cls2_vocab = get_class_vocab(2, n_grams)
  cls3_vocab = get_class_vocab(3, n_grams)
  cls4_vocab = get_class_vocab(4, n_grams)

  probs_arr = []

  cls0_prob = get_prob_class_ngram(text, cls0_vocab, n_grams,0, laplace_param)
  cls1_prob = get_prob_class_ngram(text, cls1_vocab, n_grams,1, laplace_param)
  cls2_prob = get_prob_class_ngram(text, cls2_vocab, n_grams,2, laplace_param)
  cls3_prob = get_prob_class_ngram(text, cls3_vocab, n_grams,3, laplace_param)
  cls4_prob = get_prob_class_ngram(text, cls4_vocab, n_grams,4, laplace_param)

  probs_arr.append(cls0_prob)
  probs_arr.append(cls1_prob)
  probs_arr.append(cls2_prob)
  probs_arr.append(cls3_prob)
  probs_arr.append(cls4_prob)

#   We select the class with max probability as the final output
  print(probs_arr)
  class_pred, prob_cls = get_max_ind(probs_arr)

  return class_pred, prob_cls

### Using uni-gram model to classify

In [ ]:
text = "స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటికీ కుటుంబంతో గడిపే అవకాశాన్ని ఏ మాత్రం వదులుకోవడం"
n_grams = 3
text = '<s>'*(n_grams-1) + text + '</s>'*(n_grams-1)
class_pred, prob = classify_text(text, n_grams)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

# As we can see this model has missclassfied this text

In [ ]:
input_texts=["ఈ వరల్డ్ కప్ మెగా టోర్నీలో మొదటి నుంచి అద్భుత ప్రదర్శన కనబరుస్తూ వరుస విజయాలు నమోదు చేసిన భారత జట్టు",
             "కాంగ్రెస్ ఉపాధ్యక్షుడు రాహుల్ అమెరికా అధ్యక్షుడు డొనాల్డ్ ట్రంప్ తో చర్చించారు",
             "మార్కెట్ వాస్తవిక కదలికల ఆధారంగా ఇచ్చిన సూచన ట్రేడింగ్కు ప్రారంభ స్థాయి కీలక",
             "స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటికీ కుటుంబంతో గడిపే అవకాశాన్ని ఏ మాత్రం వదులుకోవడ",
             "ఆర్థికవేత్త బెంగుళూరు ఐఐఎం రిటైర్డ్ అధ్యాపకుడు వైద్యనాథన్"
            ]

In [ ]:
for text in input_texts:
    n_grams = 2
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

In [ ]:
for text in input_texts:
    n_grams = 2
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 2)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
for text in input_texts:
    n_grams = 3
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
for text in input_texts:
    n_grams = 4
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
for text in input_texts:
    n_grams = 4
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
for text in input_texts:
    n_grams = 5
    text = '<s> '*(n_grams-1) + text + ' </s>'*(n_grams-1)
    print(text)
    class_pred, prob = classify_text(text, n_grams, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
test_sample = x_test[0]
    

In [ ]:
test_sample

In [ ]:
n_grams = 3
test_sample = '<s> '*(n_grams-1) + test_sample + ' </s>'*(n_grams-1)
print(test_sample)
class_pred, prob = classify_text(test_sample, n_grams, laplace_param = 1)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob)

In [ ]:
telugu_news_df.shape

### Using bi-gram model to classify

In [ ]:
new = "కాంగ్రెస్ ప్రభుత్వం నక్సలైట్ల పట్ల సానుభూతితో ఉందని, రాష్ట్రాన్ని నాశనం చేసిందని మోదీ చెబుతున్నారు. కాంగ్రెస్ ప్రభుత్వమే నక్సలైట్లకు మద్దతిచ్చి ఉంటే రాష్ట్ర ప్రభుత్వాన్ని కేంద్రం ఎందుకు డిస్మిస్ చేయలేదు? ఆ అధికారం వారికి ఉంది కదా? అంటే ఈ ఆరోపణల్లో నిజం లేనట్టే కదా. కేవలం ఎన్నికల సమయంలో ఇక్కడకు వచ్చి మతం పేరుతో ప్రజలను వాళ్లు రెచ్చగొడుతుంటారు.రాష్ట్రంలో ఎలాంటి ప్రభుత్వ వ్యతిరేకత లేదని, కాంగ్రెస్ ప్రభుత్వం తిరిగి అధికారంలోకి వస్తుందని రాజస్థాన్ ముఖ్యమంత్రి అశోక్ గెహ్లాట్ తెలిపారు. ప్రజలకు మంచి ప్రభుత్వాన్ని అందించామని, నీళ్లు, విద్యుత్, విద్య, ఆరోగ్యం, రోడ్ల అనుసంధానం వంటివి తమ ప్రభుత్వం కల్పించిదని చెప్పారు."

In [ ]:
'u\200c' in new

In [ ]:
inv_topic_dict

In [ ]:
for text in input_texts:
    class_pred, prob = classify_text(text, 3, laplace_param = 1)
    print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

In [ ]:
telugu_news_df[telugu_news_df['topic']== 1]['body_processed'][11]

In [ ]:
telugu_news_df

In [ ]:
text = "స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటికీ కుటుంబంతో గడిపే అవకాశాన్ని ఏ మాత్రం వదులుకోవడం"
class_pred, prob = classify_text(text, 2, laplace_param = 2)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

# The same text is now classified better with the bi-gram model

In [ ]:
# Using another text piece for the classification
text = "బాహుబలి2 సినిమా సంచలనం మెజారిటీ జనాలు మాత్రం బాహుబలి2ను ఆదరిస్తున్నారు సినీ ప్రముఖులు కూడా బాహుబలి2ను"
class_pred, prob = classify_text(text, 2, laplace_param = 2)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

### Using tri-grams to classify

In [ ]:
telugu_news_df.shape

In [ ]:
test_news_df.shape

In [ ]:
text = "బాహుబలి2 సినిమా సంచలనం మెజారిటీ జనాలు మాత్రం బాహుబలి2ను ఆదరిస్తున్నారు సినీ ప్రముఖులు కూడా బాహుబలి2ను దర్శకుడు రాజమౌళిని ప్రశంసలతో ముంచెత్తుతున్నారు"
class_pred, prob = classify_text(text, 3, laplace_param = 3)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

# As we can see, as the input text's length increases out language model's confidence in the prediction also decreses
# Here, also the text is misclassified

### Using 4-gram model 

In [ ]:
text = "బాహుబలి2 సినిమా సంచలనం మెజారిటీ జనాలు మాత్రం బాహుబలి2ను ఆదరిస్తున్నారు సినీ ప్రముఖులు కూడా బాహుబలి2ను దర్శకుడు రాజమౌళిని ప్రశంసలతో ముంచెత్తుతున్నారు"
class_pred, prob = classify_text(text, 4, laplace_param = 5)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

### Using the 5-gram model

In [ ]:
text = "బాహుబలి2 సినిమా సంచలనం మెజారిటీ జనాలు మాత్రం బాహుబలి2ను ఆదరిస్తున్నారు సినీ ప్రముఖులు కూడా బాహుబలి2ను దర్శకుడు రాజమౌళిని ప్రశంసలతో ముంచెత్తుతున్నారు"
class_pred, prob = classify_text(text, 5, laplace_param = 5)
print("Predticted class ->",inv_topic_dict[class_pred], "\nPredicted with probabilty = " ,prob*100)

**As we can see, as the length of the input sentance increses, the final probability of prediction diminishes due to the multiplication of probabilies, so we generally use the log probabilities instead of normal multiplication of probabilities.**

# **Classification of telugu text based on topic**

### Preprocessing the test dataset (removing punctuations, tokenising, etc)

In [ ]:
test_path = "../input/telugu-nlp/telugu_news/test_telugu_news.csv"

test_news_df = pd.read_csv(test_path)

del test_news_df["heading"]
del test_news_df["SNo"]
del test_news_df["date"]

test_news_df.head()

In [ ]:
y_test = test_news_df["topic"].apply(func_topic)
del test_news_df["topic"]

In [ ]:
test_news_df["body_processed"] = test_news_df["body"].str.replace('\u200c', '')
test_news_df["body_processed"] = test_news_df["body_processed"].str.replace('\n', '')
test_news_df["body_processed"] = test_news_df["body_processed"].str.replace('\t', '')
test_news_df["body_processed"] = test_news_df["body_processed"].str.replace('\xa0', '')

test_news_df["body_processed"] = test_news_df["body_processed"].apply(lambda text: remove_punctuation(text))

del test_news_df["body"]

In [ ]:
test_news_df.head()

In [ ]:
categories = [i for i in range(5)]

test_text = []
for t in test_news_df["body_processed"]:
  test_text.append(t)

In [ ]:
x_test = test_text

print(len(x_test) , len(y_test))

## Using Count Vectorizer to get the data into sk-learn's format

**Using Count Vectorizer to get the feature vectors and eliminate the stopwords (based on term and inverse document freqency and selecting the top k words in the vacabulary for model development purpose)**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
categories = [i for i in range(5)]

text_topic = []
for i in range(5):
  curr_text = ""

  for text in telugu_news_df[telugu_news_df["topic"] == i]["body_processed"]:
    curr_text += text
    curr_text += " "
    
  text_topic.append(curr_text)

In [ ]:
len(text_topic)

for i in range(5):
  print(len(text_topic[i]))

In [ ]:
from indicnlp.tokenize import indic_tokenize  


def get_all_vocab(tot_text):
  dic = {}
  for t in indic_tokenize.trivial_tokenize(tot_text): 
    if t not in dic:
      dic[t] = 1
    else:
      dic[t] += 1

  return dic 
  
tot_text = ""
for i in range(5):
  tot_text += text_topic[i]

In [ ]:
tot_vocab = get_all_vocab(tot_text)
tot_vocab = {k: v for k, v in sorted(tot_vocab.items(), key=lambda item: item[1], reverse = True)}

print(len(tot_vocab))

In [ ]:
x_train = text_topic
y_train = categories

In [ ]:
import regex 
from indicnlp.tokenize import indic_tokenize

# Using custom analyser for the count vectorizer (as telugu is an indeic language)
def custom_analyzer(text):
    words = regex.findall(r'\w{1,}', text) #extract words of at least 2 letters
    for w in words:
        yield w

In [ ]:
# We are using both uni-grams and bi-grams here to get the feature vectors

# We selected only the top 100000 words from the corpus to represent our data

count_vec = CountVectorizer(max_df = 0.75,min_df=0.0, lowercase = False , analyzer = custom_analyzer, max_features=100000, ngram_range=(1,2))

x_train_features = count_vec.fit_transform(x_train)

In [ ]:
x_train_features.toarray()

In [ ]:
x_train_features.shape

In [ ]:
# Stop words identified from the corpus by using the term-frequencies and the inverse document frequencies
len(count_vec.stop_words_)

In [ ]:
# Getting the testing data's features
x_test_features = count_vec.transform(x_test)

## Using **Multinomial Naive Baye's** classifier from sk-learn to classify the given telugu texts in the test dataset

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train_features, y_train)
MultinomialNB()

In [ ]:
print("Test score :- ", clf.score(x_test_features, y_test))

In [ ]:
from sklearn.metrics import classification_report

y_pred_test = clf.predict(x_test_features)
target_names = list(inv_topic_dict.values())

In [ ]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,average_precision_score,f1_score
confusion_mat = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(10,4))
plt.title("Confusion matrix for test data")
plt.xlabel("Predicted class")
plt.ylabel("True class")

sns.heatmap(confusion_mat, annot=True, cmap="Greens",  xticklabels = target_names,
           yticklabels=target_names)
plt.show()

In [ ]:
# Function for predicting a sample text with the trained model

def predict_text_sample(test_text, inv_topic_dict, clf, count_vec):
  test_sample = [test_text]
  x_test_sample_fetaures = count_vec.transform(test_sample)
  y_pred_test_sample = clf.predict(x_test_sample_fetaures)

  return inv_topic_dict[y_pred_test_sample[0]]

In [ ]:
input_texts=["ఈ వరల్డ్ కప్ మెగా టోర్నీలో మొదటి నుంచి అద్భుత ప్రదర్శన కనబరుస్తూ వరుస విజయాలు నమోదు చేసిన భారత జట్టు.",
             "కాంగ్రెస్ ఉపాధ్యక్షుడు రాహుల్ అమెరికా అధ్యక్షుడు డొనాల్డ్ ట్రంప్ తో చర్చించారు",
             "మార్కెట్ వాస్తవిక కదలికల ఆధారంగా ఇచ్చిన సూచన ట్రేడింగ్కు ప్రారంభ స్థాయి కీలక",
             "స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటికీ కుటుంబంతో గడిపే అవకాశాన్ని ఏ మాత్రం వదులుకోవడ",
             "ఆర్థికవేత్త బెంగుళూరు ఐఐఎం రిటైర్డ్ అధ్యాపకుడు వైద్యనాథన్"
            ]

In [ ]:
for sentence in input_texts:
    print(predict_text_sample(sentence, inv_topic_dict, clf, count_vec))
    
    

In [ ]:
# This text is about some movie news from a telugu blogsite
test_text ="ఇండియన్ స్క్రీన్ మీద పోటీ పడటం అయిపోయింది అందుకే ఇప్పుడు మన సినిమాలు ఫారిన్ రిలీజ్ లో పోటీ పడుతున్నాయి. ఇండియన్ సినిమాలు ముఖ్యంగా సౌత్ సినిమాలు రిలీజ్ అవుతున్నాయి అంటే వరల్డ్ వైడ్ మార్కెట్ ఓపెన్ అవుతుంది. తెలుగు తమిళ హిందీ భాషల సినిమాలు సబ్ టైటిల్స్ తో విధేశాల్లో కూడా రిలీజ్ అవుతున్నాయి"

print("Prediced class is " , predict_text_sample(test_text, inv_topic_dict, clf, count_vec))

In [ ]:
# This sample text is about some political news from a telugu news website (Eenadu)

test_text = "హైదరాబాద్: తెలంగాణలో సంచలనం సృష్టించిన ‘ఎమ్మెల్యేలకు ఎర’ కేసులో హైకోర్టు కీలక తీర్పు వెల్లడించింది. ఈ కేసులో ముగ్గురు నిందితుల రిమాండ్‌కు ఉన్నత న్యాయస్థానం అనుమతించింది. నిందితులు వెంటనే సైబరాబాద్ కమిషనర్‌ స్టీఫెన్‌ రవీంద్ర ఎదుట లొంగిపోవాలని ఆదేశించింది. ఒకవేళ లొంగిపోకపోతే వారిని అరెస్టు చేసి ఏసీబీ కోర్టులో హాజరుపర్చాలని.. ఆ తర్వాత రిమాండ్‌కు తరలించాలని పోలీసులను ధర్మాసనం ఆదేశించింది."
print("Prediced class is " , predict_text_sample(test_text, inv_topic_dict, clf, count_vec))

In [ ]:
# This sample text is about some sports news from a telugu news website (Eenadu)

test_text = "టీ20 ప్రప్రంచకప్‌లో టీమ్‌ఇండియా ఆడటానికి ముందు న్యూజిలాండ్‌తో వార్మప్‌ మ్యాచ్‌ జరగాల్సి ఉంది. అయితే వర్షం కారణంగా రద్దైంది. ఆ తర్వాత పాకిస్థాన్‌తో మెల్‌బోర్న్‌ వేదికగా మ్యాచ్‌కు వరుణుడి గండం ఉందని వార్తలు వచ్చాయి. దీంతో సోషల్‌ మీడియాలో మీమ్స్‌ కూడా భారీ స్థాయిలో ట్రోల్‌ అయ్యాయి. అయితే వాతావరణం అనుకూలంగా మారడంతో దాయాదుల పోరును అక్టోబర్‌ 23న వీక్షించే అవకాశం అభిమానులకు దక్కింది. ఉత్కంఠపోరులో టీమ్‌ఇండియా విజయం సాధించిన విషయం తెలిసిందే. "
print("Prediced class is " , predict_text_sample(test_text, inv_topic_dict, clf, count_vec))

In [ ]:
type(inv_topic_dict)

In [ ]:
len(inv_topic_dict)

### As we can see, using bigrams with the multinomial naive bayes classifier works absoutley well, even with text that is out of the given test corpus!

****************************************************************************


**Kindly upvote if you found this notebook helpful!  : )**